In [21]:
# Baseline
# Get list of authors in json
import json
pairs = json.load(open ("dblp/dblp_classification_all.json"))
count_non_existing = 0
authors = set()
print(len(pairs))
for pair in pairs:
    author1 = pair[0]
    author2 = pair[1]
    is_dup = pair[2]
    authors.add(author1)
    authors.add(author2)
out = open("authors.txt","w")
for a in authors:
    out.write(a+"\n")
out.close()

7155


In [96]:
# Filter the dataset and populate direct coauthors
import json
from rdflib import URIRef, Literal, Graph
from rdflib.namespace import XSD
from rdflib.plugins import sparql
pairs = json.load(open ("dblp/dblp_classification_all.json"))
count_non_existing = 0
data_set = dict()
print(len(pairs))
counter = 0
for pair in pairs:
    author1 = pair[0]
    q = sparql.prepareQuery(
        "SELECT DISTINCT ?name WHERE {{ <{}> <http://xmlns.com/foaf/0.1/name> ?name .}}".format(str(author1)))
    result = g.query(q)
    for item in result:
        author1_name =item[0].n3().replace("\"","")
    author2 = pair[1]
    q = sparql.prepareQuery(
        "SELECT DISTINCT ?name WHERE {{ <{}> <http://xmlns.com/foaf/0.1/name> ?name .}}".format(str(author2)))
    result = g.query(q)
    for item in result:
        author2_name =item[0].n3().replace("\"","")
    is_dup = pair[2]
    if counter % 100 == 0:
        print(str(counter))
    
    data_set[author1] = dict()
    data_set[author1]["name"] = author1_name
    q = sparql.prepareQuery(
        """SELECT DISTINCT ?coauthor WHERE {{
        ?paper <http://xmlns.com/foaf/0.1/maker> <{}> .
        ?paper <http://xmlns.com/foaf/0.1/maker> ?coauthor .
        }}""".format(author1))
    coauthors = g.query(q)
    q = sparql.prepareQuery(
        """SELECT DISTINCT ?conf WHERE {{
        ?paper <http://xmlns.com/foaf/0.1/maker> <{}> .
        ?paper <http://purl.org/dc/elements/1.1/publisher> ?conf .
        }}""".format(author1))
    confs = g.query(q)
    data_set[author1]["coauthors"] = set()
    data_set[author1]["confs"] = set()
    
    for coauthor in coauthors:
        data_set[author1]["coauthors"].add(str(coauthor[0]))
        
    if author1 in data_set[author1]["coauthors"]:
        data_set[author1]["coauthors"].remove(author1)
    for conf in confs:
        data_set[author1]["confs"].add(str(conf[0]))
       
    data_set[author2] = dict()
    data_set[author2]["name"] = author2_name
    q = sparql.prepareQuery(
        """SELECT DISTINCT ?coauthor WHERE {{
        ?paper <http://xmlns.com/foaf/0.1/maker> <{}> .
        ?paper <http://xmlns.com/foaf/0.1/maker> ?coauthor .
        }}""".format(author2))
    coauthors = g.query(q)
    q = sparql.prepareQuery(
        """SELECT DISTINCT ?conf WHERE {{
        ?paper <http://xmlns.com/foaf/0.1/maker> <{}> .
        ?paper <http://purl.org/dc/elements/1.1/publisher> ?conf .
        }}""".format(author2))
    confs = g.query(q)
    data_set[author2]["coauthors"] = set()
    data_set[author2]["confs"] = set()
    for coauthor in coauthors:
        data_set[author2]["coauthors"].add(str(coauthor[0]))
    if author1 in data_set[author2]["coauthors"]:
        data_set[author2]["coauthors"].remove(author1)
    for conf in confs:
        data_set[author2]["confs"].add(str(conf[0]))
    counter += 1

7155
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100


In [120]:
#Naive
counter = 1
from fuzzywuzzy import fuzz
visited = set()
cluster = dict()
c = 0
a = 0
b = 0
threshold = 70
for a1, v1 in data_set.items():
    for a2, v2 in data_set.items():
        if counter % 1000000 == 0:
            print(counter,a,b)
        if a1 != a2:
            a1_name = data_set[a1]["name"]
            a1_name = ''.join([i for i in a1_name if not i.isdigit()])
            a2_name = data_set[a2]["name"]
            a2_name = ''.join([i for i in a2_name if not i.isdigit()])
            
            if fuzz.ratio(a1_name, a2_name) > threshold:
                a += 1
                if a1 in cluster:
                    cluster[a2] = cluster[a1]
                elif a2 in cluster:
                    cluster[a1] = cluster[a2]
                else:
                    cluster[a1] = c
                    cluster[a2] = c
                    c +=1
            else:
                b += 1
                if a1 not in cluster:
                    cluster[a1] = c
                    c+=1
                if a2 not in cluster:
                    cluster[a2] = c
                    c+=1
            counter += 1


(1000000, 348, 999651)
(2000000, 959, 1999040)
(3000000, 1863, 2998136)
(4000000, 2683, 3997316)
(5000000, 3079, 4996920)
(6000000, 3444, 5996555)
(7000000, 4137, 6995862)
(8000000, 4388, 7995611)
(9000000, 4948, 8995051)
(10000000, 6036, 9993963)
(11000000, 6425, 10993574)
(12000000, 7254, 11992745)
(13000000, 7888, 12992111)
(14000000, 8173, 13991826)
(15000000, 8766, 14991233)
(16000000, 10364, 15989635)
(17000000, 11209, 16988790)
(18000000, 11503, 17988496)
(19000000, 12942, 18987057)
(20000000, 13572, 19986427)
(21000000, 14503, 20985496)
(22000000, 15706, 21984293)
(23000000, 16424, 22983575)
(24000000, 17342, 23982657)
(25000000, 17839, 24982160)
(26000000, 18198, 25981801)
(27000000, 18988, 26981011)
(28000000, 19810, 27980189)
(29000000, 20409, 28979590)
(30000000, 21267, 29978732)
(31000000, 21862, 30978137)
(32000000, 22410, 31977589)
(33000000, 23305, 32976694)
(34000000, 23740, 33976259)
(35000000, 24313, 34975686)
(36000000, 25354, 35974645)
(37000000, 26164, 36973835)
(

In [121]:
print(cluster)
cluster_to_authors = dict()
l = 0
for key, value in cluster.items():
    if value not in cluster_to_authors:
        cluster_to_authors[value] = set()
    cluster_to_authors[value].add(key)
print(l)
  

{u'http://dblp.l3s.de/d2r/resource/authors/James_M._Reecy': 0, u'http://dblp.l3s.de/d2r/resource/authors/Birgit_Korherr': 1, u'http://dblp.l3s.de/d2r/resource/authors/Alf_Ritter': 2, u'http://dblp.l3s.de/d2r/resource/authors/Lorenzo_J._Tard%C3%B3n': 3, u'http://dblp.l3s.de/d2r/resource/authors/Yael_Maon': 4, u'http://dblp.l3s.de/d2r/resource/authors/Lorenza_Giupponi': 5, u'http://dblp.l3s.de/d2r/resource/authors/Paul_Aljabar': 6, u'http://dblp.l3s.de/d2r/resource/authors/Jang_Kim': 1027, u'http://dblp.l3s.de/d2r/resource/authors/Jimmy_Shinn-Hwa_Wang': 8, u'http://dblp.l3s.de/d2r/resource/authors/Jos%C3%A9_Camargo_da_Costa': 430, u'http://dblp.l3s.de/d2r/resource/authors/Bernhard_Wiegel': 10, u'http://dblp.l3s.de/d2r/resource/authors/Khalil_Alipour': 11, u'http://dblp.l3s.de/d2r/resource/authors/Thomas_Schoberth': 3383, u'http://dblp.l3s.de/d2r/resource/authors/Ashley_McClenaghan': 12, u'http://dblp.l3s.de/d2r/resource/authors/Stavros_A._Karkanis': 2644, u'http://dblp.l3s.de/d2r/resourc

In [122]:
dups = dict()
dups_count = 0
for row in pairs:
    if row[2] == 1:
        dups[row[0]]=row[1]
        dups[row[1]]=row[0]
        dups_count+=1
print(dups_count)

2344


In [123]:
print(len(cluster.items()))

6722


In [152]:
# Testing
visited = set()
found1 = 0
found2 = 0
false1 = 0
false2 = 0
not_intersect = 0
for c in cluster_to_authors:
    for a in cluster_to_authors[c]:
        for b in cluster_to_authors[c]:
            if a not in visited and b not in visited and a != b:
                if len(max(data_set[a]["coauthors"],data_set[b]["coauthors"])) != 0:
                    intersect1 = float(len(data_set[a]["coauthors"].intersection(data_set[b]["coauthors"])))/len(max(data_set[a]["coauthors"],data_set[b]["coauthors"]))
    #                 print(intersect)
                    if intersect1 > 0.30:
#                         print(a, b)
                        if a in dups and dups[a] == b or b in dups and dups[b] == a:
                            visited.add(a)
                            visited.add(b)
    #                         print(train_set[a]["first"].intersection(train_set[b]["first"]))
                            found1 += 1
                        else:
    #                         print(a, b)
                            false1 += 1

#                 else:
#                     if len(train_set[a]["second"]) > 0 and len(train_set[b]["second"]) > 0:
#                         intersect2 = len(train_set[a]["second"].intersection(train_set[b]["second"]))/len(max(train_set[a]["second"],train_set[b]["second"]))
#                         if intersect2 > 0:
# #                             print(a, b)
#                             if a in dups and dups[a] == b or b in dups and dups[b] == a:
#                                 visited.add(a)
#                                 visited.add(b)
#                                 print(train_set[a]["second"].intersection(train_set[b]["second"]))
#                                 found2 += 1
#                             else:
# #                                 print(a, b)
#                                 false2 += 1
#                         else:
#                             not_intersect += 1
                    
                    else:
                        not_intersect += 1
precision = float(found1)/(found1+false1)
recall = float(found1)/dups_count
print("Dups count :"+str(dups_count))
print("Found first co-authors count: "+str(found1)  )
print("False first co-authors count: "+str(false1)  )
print("No intersection "+str(not_intersect))
print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("Fscore: "+str(float(2*precision*recall)/(precision+recall)))

Dups count :2344
Found first co-authors count: 386
False first co-authors count: 73
No intersection 118894
Precision: 0.840958605664
Recall: 0.164675767918
Fscore: 0.275419193721


In [151]:
# Testing
visited = set()
found1 = 0
found2 = 0
false1 = 0
false2 = 0
not_intersect = 0
for c in cluster_to_authors:
    for a in cluster_to_authors[c]:
        for b in cluster_to_authors[c]:
            if a not in visited and b not in visited and a != b:
                if fuzz.ratio(data_set[a]["name"], data_set[b]["name"]) > 90:
                    print(data_set[a]["name"],data_set[b]["name"])
#                     break
                    if a in dups and dups[a] == b or b in dups and dups[b] == a:
                        visited.add(a)
                        visited.add(b)
#                         print(train_set[a]["first"].intersection(train_set[b]["first"]))
                        found1 += 1
                    else:
#                         print(a, b)
                        false1 += 1
                                    

(u'Jie Qiu 0001', u'Jie Qiu 0002')
(u'Jie Qiu 0002', u'Jie Qiu 0001')
(u'Bard Ermentrout', u'G. Bard Ermentrout')
(u'G. Bard Ermentrout', u'Bard Ermentrout')
(u'Scott F. Smith 0003', u'Scott F. Smith 0002')
(u'Scott F. Smith 0002', u'Scott F. Smith 0003')
(u'Scott F. Smith 0001', u'Scott F. Smith 0003')
(u'Bobji Mungamuru', u'Bob Mungamuru')
(u'Bob Mungamuru', u'Bobji Mungamuru')
(u'Jo\xe3o Carlos Silva 0001', u'Jo\xe3o Carlos Silva 0002')
(u'Jo\xe3o Carlos Silva 0002', u'Jo\xe3o Carlos Silva 0001')
(u'Rita Puzmanova', u'Rita Puzmanov\xe1')
(u'Rita Puzmanov\xe1', u'Rita Puzmanova')
(u'Francesco Gagliardi 0005', u'Francesco Gagliardi 0001')
(u'Francesco Gagliardi 0001', u'Francesco Gagliardi 0005')
(u'Francesco Santini 0001', u'Francesco Santini 0002')
(u'Francesco Santini 0002', u'Francesco Santini 0001')
(u'Mohammad Taghi Hajiaghayi', u'MohammadTaghi Hajiaghayi')
(u'MohammadTaghi Hajiaghayi', u'Mohammad Taghi Hajiaghayi')
(u'Hyunchul Park 0002', u'Hyunchul Park 0001')
(u'Hyunchul Park

In [149]:
precision = float(found1)/(found1+false1)
recall = float(found1)/dups_count
print("Dups count :"+str(dups_count))
print("Found first co-authors count: "+str(found1)  )
print("False first co-authors count: "+str(false1)  )
print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("Fscore: "+str(float(2*precision*recall)/(precision+recall)))

Dups count :2344
Found first co-authors count: 83
False first co-authors count: 2624
Precision: 0.0306612486147
Recall: 0.035409556314
Fscore: 0.0328647792516


In [158]:
#Naive
counter = 1
from fuzzywuzzy import fuzz
visited = set()
cluster = dict()
c = 0
a = 0
b = 0
threshold = 90
for a1  in data_set:
    for a2 in data_set:
        if counter % 1000 == 0:
            print(counter,a1,a2)
        if a1 not in visited and a2 not in visited and a1 != a2:

            a1_name = data_set[a1]["name"]
            a1_name = ''.join([i for i in a1_name if not i.isdigit()])
            a2_name = data_set[a2]["name"]
            a2_name = ''.join([i for i in a2_name if not i.isdigit()])
            
            if fuzz.ratio(a1_name, a2_name) > threshold:
                if a in dups and dups[a] == b or b in dups and dups[b] == a:
                    visited.add(a)
                    visited.add(b)
#                         print(train_set[a]["first"].intersection(train_set[b]["first"]))
                    found1 += 1
                else:
#                         print(a, b)
                    false1 += 1
            

In [157]:
precision = float(found1)/(found1+false1)
recall = float(found1)/dups_count
print("Dups count :"+str(dups_count))
print("Found first co-authors count: "+str(found1)  )
print("False first co-authors count: "+str(7155-2344)  )
print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("Fscore: "+str(float(2*precision*recall)/(precision+recall)))

Dups count :2344
Found first co-authors count: 386
False first co-authors count: 4811
Precision: 0.00560411161764
Recall: 0.164675767918
Fscore: 0.0108393473927
